<a href="https://colab.research.google.com/github/dropbotsine/mon-bot-dropping-odds/blob/main/BotFixSine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 13.9 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=7747266ab6dd66166e149afa552736246cb0518a366e28f74efb6efa0bb4f146
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [ ]:
import re
import nest_asyncio
import asyncio
from collections import Counter
from telethon import TelegramClient
from datetime import datetime, timedelta

nest_asyncio.apply()

# 🔧 Paramètres modifiables
INCLUDE_FOOTBALL = True
ALERT_THRESHOLD = 25
CHANNELS = ["@dropping_odds_alert", "@dropping_odds_alert_1h"]
MESSAGE_LIMIT = 3000
START_DATE = datetime(2025, 2, 7)

# 🏆 Émojis pour les sports
SPORT_EMOJIS = {
    "basketball": "🏀",
    "tennis": "🎾",
    "hockey": "🏒",
    "volleyball": "🏐",
    "baseball": "⚾",
    "handball": "🤾",
    "rugby": "🏉",
    "table tennis": "🏓",
    "badminton": "🏸",
    "american football": "🏈",
    "football": "⚽",
}

DEFAULT_EMOJI = "🏅"

API_ID = 22491096
API_HASH = "770f00ae81b7347f7b49a32522090219"
SESSION_NAME = "telegram_session"

client = TelegramClient(SESSION_NAME, API_ID, API_HASH)

def clean_prono(prono_text):
    match = re.match(r"(.+?)\s\*\*\d+\.\d+\*\*\s⇩\d+%", prono_text)
    return match.group(1).strip() if match else prono_text.strip()

def extract_last_odds(prono_text):
    match = re.search(r"\*\*(\d+\.\d+)\*\*\s⇩(\d+)%", prono_text)
    return (float(match.group(1)), int(match.group(2))) if match else (None, None)

def extract_match_link(match_text):
    match = re.search(r"\[(.*?)\]\((https?://[^\s]+)\)", match_text)
    return match.groups() if match else (match_text, None)

async def process_channel(channel):
    channel_match_info = {}

    async for message in client.iter_messages(channel, limit=MESSAGE_LIMIT):
        if message.text:
            match_ids = re.findall(r"#([A-Za-z0-9]{8})", message.text)
            date_time_match = re.search(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2})", message.text)
            message_datetime = datetime.strptime(date_time_match.group(), "%Y-%m-%d %H:%M") + timedelta(hours=1) if date_time_match else None

            if message_datetime and message_datetime < START_DATE:
                continue

            lines = message.text.splitlines()
            sport_field = lines[1].strip().lower() if len(lines) >= 2 else "N/A"

            if not INCLUDE_FOOTBALL and "football" in sport_field:
                continue

            match_field = lines[3].strip() if len(lines) >= 4 else "N/A"
            match_name, match_link = extract_match_link(match_field)

            prono_field = lines[6].strip() if len(lines) >= 7 else "N/A"
            clean_prono_text = clean_prono(prono_field)
            odds_value, odds_percentage = extract_last_odds(prono_field)
            sport_emoji = SPORT_EMOJIS.get(sport_field, DEFAULT_EMOJI)

            for match_id in match_ids:
                if match_id not in channel_match_info:
                    channel_match_info[match_id] = {
                        'count': 1,
                        'datetime': message_datetime,
                        'sport': sport_field,
                        'match_name': match_name,
                        'match_link': match_link,
                        'prono_freq': Counter({clean_prono_text: 1}),
                        'latest_odds': [(clean_prono_text, odds_value, odds_percentage)],
                        'emoji': sport_emoji
                    }
                else:
                    channel_match_info[match_id]['count'] += 1
                    channel_match_info[match_id]['prono_freq'][clean_prono_text] += 1
                    channel_match_info[match_id]['latest_odds'].append((clean_prono_text, odds_value, odds_percentage))

                    if message_datetime is not None:
                        current_dt = channel_match_info[match_id]['datetime']
                        if current_dt is None or message_datetime > current_dt:
                            channel_match_info[match_id]['datetime'] = message_datetime
                            channel_match_info[match_id]['sport'] = sport_field
                            channel_match_info[match_id]['match_name'] = match_name
                            channel_match_info[match_id]['match_link'] = match_link
                            channel_match_info[match_id]['emoji'] = sport_emoji

        await asyncio.sleep(0)
    return channel_match_info

async def get_match_alerts():
    print("🔌 Connexion en cours... 🌐")
    await client.start()
    print("✅ Connecté avec succès !")

    print("🔍 Scan en cours...")
    tasks = [process_channel(channel) for channel in CHANNELS]
    results = await asyncio.gather(*tasks)

    global_match_info = {}
    for channel_info in results:
        for match_id, info in channel_info.items():
            if match_id not in global_match_info:
                global_match_info[match_id] = info
            else:
                global_match_info[match_id]['count'] += info['count']
                for prono, freq in info['prono_freq'].items():
                    global_match_info[match_id]['prono_freq'][prono] += freq

                global_match_info[match_id]['latest_odds'].extend(info['latest_odds'])

                if info['datetime'] is not None:
                    current_dt = global_match_info[match_id]['datetime']
                    if current_dt is None or info['datetime'] > current_dt:
                        global_match_info[match_id].update({
                            'datetime': info['datetime'],
                            'sport': info['sport'],
                            'match_name': info['match_name'],
                            'match_link': info['match_link'],
                            'emoji': info['emoji']
                        })

    sorted_matches = sorted(global_match_info.items(), key=lambda x: x[1]['datetime'] if x[1]['datetime'] else datetime.min)

    filtered_matches = [info for match_id, info in sorted_matches if info['count'] >= ALERT_THRESHOLD]

    print(f"\n📢 Nombre de matchs filtrés : {len(filtered_matches)}\n")

    print("\n📢 MATCHS À VENIR")
    print(f"📅 À PARTIR DU {START_DATE.strftime('%Y-%m-%d')}")
    print(f"🔔 ALERTES MINIMUM : {ALERT_THRESHOLD}")
    print("")

    for match_id, info in sorted_matches:
        if info['count'] >= ALERT_THRESHOLD:
            dt_str = info['datetime'].strftime("%Y-%m-%d %H:%M") if info['datetime'] else "N/A"
            sport_emoji = info['emoji']
            prono_most_common = info['prono_freq'].most_common(1)[0][0] if info['prono_freq'] else "N/A"

            # Récupérer la cote et le pourcentage les plus récents pour ce prono
            latest_odds_for_prono = [(odds, perc) for prono, odds, perc in info['latest_odds'] if prono == prono_most_common]
            if latest_odds_for_prono:
                recent_odds = latest_odds_for_prono[-1]
                last_odds_str = f"{recent_odds[0]} ⇩{recent_odds[1]}%" if recent_odds[0] is not None else "N/A"
            else:
                last_odds_str = "N/A"

            print("")
            print(f"#{match_id}")
            print(f"📊 Alertes : {info['count']}")
            print(f"📅 {dt_str}")
            print(f"{sport_emoji} {info['sport'].capitalize()}")
            print(f"📍 [{info['match_name']}]({info['match_link']})" if info['match_link'] else f"📍 {info['match_name']}")
            print(f"🎯 {prono_most_common} ({last_odds_str})")
            print("")
            print("-" * 40)

    print("🔌 Déconnexion en cours...")
    await client.disconnect()
    print("✅ Déconnecté avec succès !")

asyncio.run(get_match_alerts())


🔌 Connexion en cours... 🌐
Please enter your phone (or bot token): +33670256370
Please enter the code you received: 20595
Signed in successfully as SINEzer; remember to not break the ToS or you will risk an account ban!
✅ Connecté avec succès !
🔍 Scan en cours...

📢 Nombre de matchs filtrés : 4


📢 MATCHS À VENIR
📅 À PARTIR DU 2025-02-07
🔔 ALERTES MINIMUM : 25


#Im1i67gT
📊 Alertes : 35
📅 2025-02-07 00:30
⚽ Football
📍 [Estudiantes Merida - Monagas](https://www.oddsportal.com/football/venezuela/liga-futve/estudiantes-merida-monagas-Im1i67gT)
🎯 AH -1.25 1 (2.27 ⇩33%)

----------------------------------------

#dCPrGrEc
📊 Alertes : 35
📅 2025-02-07 03:00
⚽ Football
📍 [Club America W - Mazatlan FC W](https://www.oddsportal.com/football/mexico/liga-mx-women/club-america-mazatlan-fc-dCPrGrEc)
🎯 O/U 5.5 Over (1.89 ⇩31%)

----------------------------------------

#zR0TPMun
📊 Alertes : 28
📅 2025-02-07 12:00
⚽ Football
📍 [Sigma Olomouc B - Zlin B](https://www.oddsportal.com/football/world/club-fri